In [1]:
#import statements here
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import functions as sf
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import RegexTokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat, col, lit, lower
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.sql.functions import concat_ws
from  pyspark.sql.functions import abs
# seting master("local[*]") enables multicore processing on all available logical cores on your machine
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml import evaluation
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import evaluation
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [2]:
import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
#load the files
airlines_df = spark.read.csv(get_training_filename('airlines.csv'), header=True, inferSchema=True)
airports_df = spark.read.csv(get_training_filename('airports.csv'), header=True, inferSchema=True)
flights_df = spark.read.csv(get_training_filename('flights.csv'), header=True, inferSchema=True)

In [4]:
#EDA

shape = ((flights_df.count(), len(flights_df.columns)))  #prints the shape
print('The shape of flights_df:', shape)

print(flights_df.select([count(when(isnull(c), c)).alias(c) for c in flights_df.columns])) # prints the null values in the columns

print(flights_df.printSchema())  #shows the schema of the datasset

flights_df_sample = flights_df.sample(True, 0.5, 42)
flights_df_sample_pandas = flights_df_sample.toPandas() #create sample for being able to create proper visualizations on the dataset

#create visualizations for relevant features to understand the data
sns.distplot(flights_df_sample_pandas['MONTH'])
display()

sns.distplot(flights_df_sample_pandas['DAY'])
display()


sns.distplot(flights_df_sample_pandas['DAY_OF_WEEK'])
display()


flights_df_sample_pandas['AIRLINE'].value_counts().plot(kind='bar')
display()

airline_count  = flights_df_sample_pandas['AIRLINE'].value_counts()
airline_count = airline_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(airline_count.index, airline_count.values, alpha=0.8)
display()



In [5]:
#create visualizations for relevant features to understand the data
oa_count  = flights_df_sample_pandas['ORIGIN_AIRPORT'].value_counts()
oa_count = oa_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(oa_count.index, oa_count.values, alpha=0.8)
display()

da_count  = flights_df_sample_pandas['DESTINATION_AIRPORT'].value_counts()
da_count = da_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(da_count.index, da_count.values, alpha=0.8)
display()


sns.distplot(flights_df_sample_pandas['SCHEDULED_DEPARTURE'])
display()


sns.distplot(flights_df_sample_pandas['DEPARTURE_DELAY'])
display()

In [6]:
#create visualizations for relevant features to understand the data
sns.distplot(flights_df_sample_pandas['DISTANCE'])
display()

sns.distplot(flights_df_sample_pandas['SCHEDULED_ARRIVAL'])
display()

sns.barplot(x="MONTH", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()


df3 = flights_df.groupBy('MONTH').avg('DEPARTURE_DELAY').toPandas()
df3
fig4 = sns.barplot(x = 'MONTH', y = 'avg(DEPARTURE_DELAY)', data = df3)
display()

sns.barplot(x="AIRLINE", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()

sns.barplot(x="SCHEDULED_DEPARTURE", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()


sns.barplot(x="ORIGIN_AIRPORT", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()


In [7]:
#create visualizations for relevant features to understand the data
sns.barplot(x="AIRLINE", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()


sns.barplot(x="DAY_OF_WEEK", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()


sns.barplot(x="DAY_OF_WEEK", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

sns.barplot(x="DAY", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()

sns.barplot(x="DAY", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

sns.barplot(x="MONTH", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()

In [8]:
#DATA PREPARATION: now we will process the dataset and convert into a usable dataset

#create dataframe with columns we will consider
flights_df = flights_df.select('MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'CANCELLED')  

#remove columns with cancel = 0 
flights_df = flights_df.filter((fn.col('CANCELLED')==0))

#set up a threshold for the delay time
flights_df = flights_df.withColumn("Flight_Delayed", fn.when(fn.col("DEPARTURE_DELAY")<15, 0).otherwise(1))

#create buckets for distance as long, short and medium
from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import udf
from pyspark.sql.types import *
bucketizer = Bucketizer(splits=[ 0, 100, 1000, float('Inf') ],inputCol="DISTANCE", outputCol="Distance_Bucket")
flights_df = bucketizer.setHandleInvalid("keep").transform(flights_df)
t = {0.0:"Short", 1.0: "Medium", 2.0:"Long"}
udf_foo = udf(lambda x: t[x], StringType())
flights_df = flights_df.withColumn("Flight_Distance", udf_foo("Distance_Bucket"))

#perform string index, which prevents creation of new columns for the dummy variables and perform one hot encoding
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

indexer = StringIndexer(inputCol="AIRLINE", outputCol="Airline_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)


encoder = OneHotEncoder(inputCol="Airline_Numeric", outputCol="Airline_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="OA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="OA_Numeric", outputCol="Origin_Airport_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="DA_Numeric", outputCol="Destination_Airport_OHE")
flights_df= encoder.transform(flights_df)

flights_df = flights_df.drop('ARRIVAL_DELAY')

In [9]:
#that dataset is imbalanced

#perform undersampling for the preparation of the data
#split the train test data
training_df, testing_df = flights_df.randomSplit([0.9, 0.1], seed=5)

#create major and minor DF 
major_df = training_df.filter(col("Flight_Delayed") == 0)
minor_df = training_df.filter(col("Flight_Delayed") == 1)
ratio = major_df.count()/minor_df.count()

#create the balanced dataset for training the model
sampled_majority_df = major_df.sample(False, 1.2/ratio, seed=5)
combined_df_2 = sampled_majority_df.unionAll(minor_df)

#shows the count of balanced dataframe
combined_df_2.groupBy('Flight_Delayed').agg(fn.count('*')).show()

+---------+--------+
CANCELLED|count(1)|
+---------+--------+
 0| 5729195|
+---------+--------+

+--------------+--------+
Flight_Delayed|count(1)|
+--------------+--------+
 1| 949914|
 0| 1139995|
+--------------+--------+

In [10]:
#implementation of prepared and undersampled dataset for the model implementation

#create a vector assembler
va = VectorAssembler(
    inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED"], outputCol="features")

#train a logistic model
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.2, elasticNetParam=0.1, threshold=0.45, thresholds=[0.55, 0.45])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)

#logistic model metric evaluation

#AUC score
bce = BinaryClassificationEvaluator(labelCol='Flight_Delayed', metricName='areaUnderROC')
score_auc=bce.evaluate(lr_transform)

#Recall
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
score_recall=lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

#Precision
lr_evaluator_precision = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedPrecision")
score_precision=lr_evaluator_precision.evaluate(lr_model.transform(testing_df))

#accuracy
lr_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="accuracy")
score_accuracy=lr_evaluator_accuracy.evaluate(lr_model.transform(testing_df))

#f1 
lr_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")
score_f1=lr_evaluator_accuracy.evaluate(lr_model.transform(testing_df))

#shows the scatterplot
plt.figure(figsize=(10,6))
plt.plot([0, 1], [0, 1], 'r--')
plt.scatter(lr_model.stages[-1].summary.roc.select('FPR').collect(),
            lr_model.stages[-1].summary.roc.select('TPR').collect())
plt.title('ROC Scatter Plot : TPR/FPR')
plt.xlabel('FPR')
plt.ylabel('TPR')
display()

In [11]:
#
scores = [score_auc, score_recall, score_accuracy, score_f1, score_precision]
metricName = ['AUC', 'Recall', 'Accuracy', 'f1', 'Precision']
metric_df = pd.DataFrame(zip(metricName,scores),index=[1,2,3,4,5],columns=['Metric Name', 'Score'])
metric_df.head(5)

,Metric Name,Score
1,AUC,0.632220
2,Recall,0.569218
3,Accuracy,0.569218
4,f1,0.617380
5,Precision,0.757864


In [12]:
#Confusion matrix of logistic regression

y_true = lr_transform.select(['Flight_Delayed']).collect()
y_pred = lr_transform.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.87 0.55 0.68 467809
 1 0.25 0.65 0.36 105586

 micro avg 0.57 0.57 0.57 573395
 macro avg 0.56 0.60 0.52 573395
weighted avg 0.76 0.57 0.62 573395

Out[23]: array([[258188, 209621],
 [ 37387, 68199]])

In [13]:
#train the GBT model on the balanced dataset
gbt = GBTClassifier(featuresCol='features', labelCol='Flight_Delayed', maxIter=5, maxBins=15, stepSize=0.08, maxDepth=4)
gbt_pipeline = Pipeline(stages=[va, gbt])
gbt_model = gbt_pipeline.fit(combined_df_2)
gbt_transform = gbt_model.transform(testing_df)

#evaluation metric for GBT
gbt_bce = BinaryClassificationEvaluator(labelCol='Flight_Delayed', metricName='areaUnderROC')
score_auc=gbt_bce.evaluate(gbt_transform)

#f1
gbt_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")
score_f1=gbt_evaluator_f1.evaluate(gbt_transform)

#accuracy
gbt_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="accuracy")
score_accuracy=gbt_evaluator_accuracy.evaluate(gbt_transform)

#precision
gbt_evaluator_precision = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedPrecision")
score_precision=gbt_evaluator_precision.evaluate(gbt_transform)

#recall
gbt_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
score_recall=gbt_evaluator_recall.evaluate(gbt_transform)

#dataframe for the evaluation metric
scores = [score_auc, score_recall, score_accuracy, score_f1, score_precision]
metricName = ['AUC', 'Recall', 'Accuracy', 'f1', 'Precision']

metric_df = pd.DataFrame(zip(metricName,scores),index=[1,2,3,4,5],columns=['Metric Name', 'Score'])
metric_df

,Metric Name,Score
1,AUC,0.658491
2,Recall,0.661530
3,Accuracy,0.661530
4,f1,0.694910
5,Precision,0.758869


In [14]:
#Confusion matrix of logistic regression

y_true = gbt_transform.select(['Flight_Delayed']).collect()
y_pred = gbt_transform.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.87 0.69 0.77 467809
 1 0.28 0.53 0.37 105586

 micro avg 0.66 0.66 0.66 573395
 macro avg 0.57 0.61 0.57 573395
weighted avg 0.76 0.66 0.69 573395

Out[28]: array([[323286, 144523],
 [ 49554, 56032]])

In [15]:
#create a further sample due to computation limit
sample = combined_df_2.sample(True, 0.2,seed=5)

#uncomment below comment if the error says label exists and run

# flights_df = flights_df.drop('label')
# training_df = training_df.drop('label')
# testing_df = testing_df.drop('label')
# sample = sample.drop('label')

features = sample.columns
sample = sample.select(col("Flight_Delayed").alias("label"), *features)

#perform train test to get the testing dataset
training_df, testing_df = flights_df.randomSplit([0.9, 0.1], seed=5)

#create vectorassembler 
va = VectorAssembler(inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED"], outputCol="unscaled_features")

#standardize the input
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")

#train model and create pipeline with crossvalidation
rf = RandomForestClassifier(featuresCol='features', labelCol='label')
rf_pipeline = Pipeline(stages=[va,standardScaler, rf])


paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [3,4]) \
    .addGrid(rf.numTrees, [100,150]) \
    .addGrid(rf.impurity,["Gini"]) \
    .addGrid(rf.featureSubsetStrategy,["auto", "sqrt"]) \
    .addGrid(rf.maxBins,[100]) \
    .build()
crossval_rf = CrossValidator(estimator=rf_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

cvModel = crossval_rf.fit(sample)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [16]:
#evaluation metric for random forest
testing_df = testing_df.select(col("Flight_Delayed").alias("label"), *features)
prediction = cvModel.transform(testing_df)

#AUC evaluation
bce = BinaryClassificationEvaluator(labelCol='label')
score_auc = bce.evaluate(prediction, {bce.metricName: "areaUnderROC"} )

#weighted precision
mce = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedPrecision")
score_precision = mce.evaluate(cvModel.transform(testing_df))

#weighted Recall
mce = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
score_recall = mce.evaluate(cvModel.transform(testing_df))

#Accuracy
mce = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="accuracy")
score_accuracy = mce.evaluate(cvModel.transform(testing_df))

#f1
mce = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")
score_f1= mce.evaluate(cvModel.transform(testing_df))

#dataframe for the evaluation metric
scores = [score_auc, score_recall, score_accuracy, score_f1, score_precision]
metricName = ['AUC', 'Recall', 'Accuracy', 'f1', 'Precision']

metric_df = pd.DataFrame(zip(metricName,scores),index=[1,2,3,4,5],columns=['Metric Name', 'Score'])
metric_df

,Metric Name,Score
1,AUC,0.650553
2,Recall,0.815858
3,Accuracy,0.815858
4,f1,0.733124
5,Precision,0.665625


In [17]:
cols = ["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL"]

randomForest_model = cvModel.bestModel.stages[-1]
RF_df = pd.DataFrame(list(zip(cols, randomForest_model.featureImportances.toArray())),
             columns = ['column', 'weight']).sort_values('weight', ascending=False)
RF_df

,column,weight
8,SCHEDULED_ARRIVAL,0.066453
0,MONTH,0.054460
3,Airline_OHE,0.045509
4,Origin_Airport_OHE,0.037430
6,SCHEDULED_DEPARTURE,0.015556
1,DAY,0.008425
2,DAY_OF_WEEK,0.006897
5,Destination_Airport_OHE,0.001321
7,Distance_Bucket,0.000245
